In [ ]:
!pip install tensorflow_io==0.23.1 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 23.1 MB 1.5 MB/s 
     |████████████████████████████████| 2.1 MB 57.4 MB/s 
  Attempting uninstall: tensorflow-io-gcs-filesystem
    Found existing installation: tensorflow-io-gcs-filesystem 0.26.0
    Uninstalling tensorflow-io-gcs-filesystem-0.26.0:
      Successfully uninstalled tensorflow-io-gcs-filesystem-0.26.0


In [ ]:
import tensorflow as tf 
import tensorflow_io as tfio
import matplotlib.pyplot as plt 
import os 

In [ ]:
!cd '/content/drive/MyDrive/Data'

In [ ]:
!unzip '/content/drive/MyDrive/archive(1).zip'

In [ ]:
def load_wav_16k_mono(file_name):
  file_contents=tf.io.read_file(file_name)
  wav,sample_rate=tf.audio.decode_wav(file_contents,desired_channels=1)
  wav=tf.squeeze(wav,axis=-1)
  sample_rate=tf.cast(sample_rate,dtype=tf.int64)
  wav=tfio.audio.resample(wav,rate_in=sample_rate,rate_out=16000)
  return wav 

In [ ]:
yes=os.path.join('/content/data','Parsed_Capuchinbird_Clips','XC114132-4.wav')
no=os.path.join('/content/data','Parsed_Not_Capuchinbird_Clips','Crickets-chirping-at-night-3.wav')

In [ ]:
Neg=os.path.join('/content/data','Parsed_Not_Capuchinbird_Clips')
Pos=os.path.join('/content/data','Parsed_Capuchinbird_Clips')

In [ ]:
pos=tf.data.Dataset.list_files(Pos+'/*.wav')
neg=tf.data.Dataset.list_files(Neg+'/*.wav')

In [ ]:
positives = tf.data.Dataset.zip((pos, tf.data.Dataset.from_tensor_slices(tf.ones(len(pos)))))
negatives = tf.data.Dataset.zip((neg, tf.data.Dataset.from_tensor_slices(tf.zeros(len(neg)))))
data = positives.concatenate(negatives)

In [ ]:
length=[]
for File in os.listdir(os.path.join('/content/data','Parsed_Capuchinbird_Clips')):
  tensor_wav=load_wav_16k_mono(os.path.join('/content/data','Parsed_Capuchinbird_Clips',File))
  length.append(len(tensor_wav))

In [ ]:
maxLen=tf.math.reduce_max(length)
meanLen=tf.math.reduce_mean(length)
minLen=tf.math.reduce_min(length)

In [ ]:
# Preprocessing 
def preprocess(file_name,label):
  wav=load_wav_16k_mono(file_name)
  wav=wav[:48000]
  zero_padding = tf.zeros(48000 -tf.shape(wav), dtype=tf.float32)
  wav=tf.concat([zero_padding,wav],0)
  spectrogram = tf.signal.stft(wav, frame_length=320, frame_step=32)
  spectrogram = tf.abs(spectrogram)
  spectrogram = tf.expand_dims(spectrogram, axis=2)
  return spectrogram,label

In [ ]:
data = data.map(preprocess)
data = data.cache()
data = data.shuffle(buffer_size=1000)
data = data.batch(16)
data = data.prefetch(8)

In [ ]:
train = data.take(36)
test = data.skip(36).take(15)

In [ ]:
from tensorflow.keras.layers import  Dense, Conv2D, Flatten 
from tensorflow.keras.models import Sequential

In [ ]:
model = Sequential()
model.add(Conv2D(16, (3,3), activation='relu', input_shape=(1491, 257,1)))
model.add(Conv2D(16, (3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


model.compile('Adam', loss='BinaryCrossentropy', metrics=[tf.keras.metrics.Recall(),tf.keras.metrics.Precision()])

In [ ]:
hist = model.fit(train, epochs=4, validation_data=test)

In [ ]:
model.save('audio_model.h5')